# NPZD slab model

For the first model example I will need:
- Forcing input
- Forcing_fluxes:
    - mixing
    - sinking
- 4 components (single) N, P, Z, D
- Fluxes:
TODO: GO OVER THE equations! Base model structure in math!
    - Grazing - linked to all NPZD
    - PhytoplanktonGrowth
        - Light - Dependence
        - Nutrients - Dependence
        - Temp - Dependence
    - mortality
    - remineralisation
    - higher order pred, etc

In [1]:
import numpy as np
import xsimlab as xs

In [2]:
import os,sys,inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import phydra


from phydra.utility.xsimlabwrappers import phydra_setup

In [3]:
from phydra.processes.main import Grid0D, Boundary0D
from phydra.processes.components import Component, Time, make_Component
from phydra.processes.environments import BaseEnvironment, Slab
from phydra.processes.fluxes import Flux, LimitedGrowth
from phydra.processes.forcingfluxes import Mixing, Sinking, Upwelling, make_FX_flux
from phydra.processes.forcing import SinusoidalMLD, ConstantN0
from phydra.processes.gekkocontext import GekkoContext, GekkoSolve, InheritGekkoContext

In [4]:
from phydra.processes.forcing import NutrientForcing, MLDForcing

In [5]:
print('xarray-simlab version: ', xs.__version__)
print('phydra version: ', phydra.__version__)

xarray-simlab version:  0.4.1
phydra version:  0+untagged.71.g1323c22.dirty


In [6]:
model = xs.Model({
    'time':Time, 'grid':Grid0D, 'ibcs':Boundary0D, 
    
    'context':GekkoContext,
                  
    'N':make_Component('Nutrient','N'), 'P':make_Component('Phytoplankton','P'),
    'Z':make_Component('Zooplankton','Z'), 'D':make_Component('Detritus','D'),
    'Env':Slab, 
    
    # Fluxes
    #'P_Growth':LimitedGrowth, 
    
    # ForcingFluxes
    'Sinking':make_FX_flux(Sinking, 'S'),
    'Upwelling':make_FX_flux(Upwelling, 'U'),
    
    'MLD':SinusoidalMLD, 
    'N0':ConstantN0, 
    
    'solver':GekkoSolve
})

model

Sinking_S
<class 'phydra.processes.forcingfluxes.Sinking'> S
Upwelling_U
<class 'phydra.processes.forcingfluxes.Upwelling'> U


<xsimlab.Model (14 processes, 17 inputs)>
context
time
    days            [in] ('time',) time in days
grid
    length          [in]
    shape           [in]
ibcs
    ibc             [in] Initial Boundary Conditions
N
    init            [in]
    dim             [in]
P
    dim             [in]
    init            [in]
Z
    dim             [in]
    init            [in]
D
    init            [in]
    dim             [in]
MLD
N0
    constant_N0     [in] value of the constant Nutrient forcing
Upwelling
    C_labels        [in] U label of component(s) that grows
    kappa           [in] constant mixing coefficient
Sinking
    kappa           [in] constant mixing coefficient
    C_labels        [in] S label of component(s) that grows
Env
solver

In [7]:
model.index_vars 


[('time', 'time'),
 ('N', 'N'),
 ('P', 'P'),
 ('Z', 'Z'),
 ('D', 'D'),
 ('Env', 'components'),
 ('Env', 'forcingfluxes'),
 ('Sinking', 'S'),
 ('Upwelling', 'U')]

In [8]:
in_ds = phydra_setup(
    model=model,
    input_vars={
        'time__days': ('time', np.arange(0, 365*3)), #365*
        
        'N':{'dim':1,'init':1}, 
        'P':{'dim':1,'init':1},
        'Z':{'dim':1,'init':1},
        'D':{'dim':2,'init':1},
        
        #'P_Growth':{'mu':1.,
        #       'C_label':'P',
        #       'R_label':'N',
        #       'halfsat':0.5},
        
        'Sinking':{'kappa':0.1,
               'C_labels':['D','P']},
        
        'Upwelling':{'kappa':0.1,
               'C_labels':['N']},
        
        'N0__constant_N0':2.
    },
    output_vars={
        'Env__comp_output':None,
        'Env__fxflux_output':None,
    }
                 )

In [9]:
with model:
    out_ds = in_ds.xsimlab.run()

Initializing Gekko Context
Initializing Model Time
dimensions of component N are initialized at 1
Initializing component  N ['N']
FULLDIMS [[0.]]
[[1]]
dimensions of component P are initialized at 1
Initializing component  P ['P']
FULLDIMS [[0.]]
[[1]]
dimensions of component Z are initialized at 1
Initializing component  Z ['Z']
FULLDIMS [[0.]]
[[1]]
dimensions of component D are initialized at 2
Initializing component  D ['D-0', 'D-1']
FULLDIMS [[0. 0.]]
[[1 1]]
forcing flux Upwelling is initialized at ['N']
['Upwelling-N-[0.]']
Initializing forcing flux Upwelling for components ['N']
N initFXflux [[list([])]]
N initFXflux [[list([0])]]
forcing flux Sinking is initialized at ['D' 'P']
['Sinking-D-[0. 0.]', 'Sinking-P-[0.]']
Initializing forcing flux Sinking for components ['D' 'P']
D initFXflux [[list([]) list([])]]
D initFXflux [[list([0]) list([0])]]
P initFXflux [[list([])]]
P initFXflux [[list([0])]]
hello there


Initializing Environment: 
 components:['N', 'P', 'Z', 'D-0', 'D-1

ValueError: conflicting sizes for dimension 'forcingfluxes': length 3 on 'forcingfluxes' and length 4 on 'Env__fxflux_output'

In [ ]:
out_ds

In [ ]:
out_ds.Env__comp_output.plot.line(x='time')

In [ ]:
out_ds.Env__fxflux_output.plot.line(x='time')